In [6]:
# Connect to the database
import sqlalchemy
from sqlalchemy import create_engine

engine = create_engine('postgresql+psycopg2://ctang:@localhost:5432/ctang', encoding='utf-8') # no password
connection = engine.connect()

In [10]:
# Set up the tables
from sqlalchemy import MetaData
from sqlalchemy import Table, Column, Enum, Boolean, Integer, Numeric, Text

metadata = MetaData()

orders = Table('orders', metadata,
    Column('order_id', Integer(), autoincrement=True, primary_key=True),
    Column('user_id', Integer(), autoincrement=True),
    Column('order_eval_set', Enum('prior', 'train', 'test')),
    Column('order_number', Integer()),
    Column('order_dow', Integer()),
    Column('order_hour_of_day', Integer()),
    Column('days_since_prior', Numeric(), nullable=True)
)

aisles = Table('aisles', metadata,
    Column('aisle_id', Integer(), autoincrement=True, primary_key=True),
    Column('aisle', Text)
)

departments = Table('departments', metadata,
    Column('department_id', Integer(), autoincrement=True, primary_key=True),
    Column('department', Text)
)

products = Table('products', metadata,
    Column('product_id', Integer(), autoincrement=True, primary_key=True),
    Column('product_name', Text()),
    Column('aisle_id', ForeignKey('aisles.aisle_id')),
    Column('department_id', ForeignKey('departments.department_id'))
)

order_products__train = Table('order_products__train', metadata,
    Column('order_id', ForeignKey('orders.order_id')),
    Column('product_id', ForeignKey('products.product_id')),
    Column('add_to_cart_order', Integer(), autoincrement=True),
    Column('reordered', Boolean())
)

In [11]:
from sqlalchemy.sql import select, func, desc

# Top 10 departments with the most product listings
columns = [departments.c.department, func.count(products.c.product_id).label('product_count')]
s = select(columns) \
    .select_from(departments.join(products)) \
    .group_by(departments.c.department) \
    .order_by(desc('product_count'))

rp = connection.execute(s)
print "%20s | %s" % ('department', 'number of products')
print '-' * 50
for record in rp:
    print "%20s | %s" % (record.department, record.product_count)

          department | number of products
--------------------------------------------------
       personal care | 6563
              snacks | 6264
              pantry | 5371
           beverages | 4365
              frozen | 4007
          dairy eggs | 3449
           household | 3085
        canned goods | 2092
     dry goods pasta | 1858
             produce | 1684
              bakery | 1516
                deli | 1322
             missing | 1258
       international | 1139
           breakfast | 1115
              babies | 1081
             alcohol | 1054
                pets | 972
        meat seafood | 907
               other | 548
                bulk | 38


In [102]:
# # /* Top 10 most ordered items */
# columns = [order_products__train.c.product_id, func.count().label('num_ordered')]
# product_counts = select(columns) \
#     .select_from(order_products__train) \
#     .group_by(order_products__train.c.product_id) \
#     .order_by(desc('num_ordered')) \
#     .limit(10) \
#     .subquery('product_counts')

# col2 = [products.c.product_name, product_counts.c.num_ordered]
# r = select(col2).select_from(product_counts).join(products).limit(10)
# print str(product_counts)
# rp = connection.execute(s)
# for record in rp:
#     print record
    
# # SELECT p.product_name, o.num_ordered, a.aisle
# #   FROM (SELECT o.product_id as product_id, COUNT(*) AS num_ordered
# #           FROM order_products__train o
# #       GROUP BY o.product_id
# #       ORDER BY num_ordered DESC) AS o
# #   JOIN products p ON p.product_id = o.product_id
# #   JOIN aisles a ON p.aisle_id = a.aisle_id
# #  LIMIT 10;

In [1]:
# Connect to the database
import sqlalchemy
from sqlalchemy import create_engine

engine = create_engine('postgresql+psycopg2://ctang:@localhost:5432/ctang', encoding='utf-8') # no password
connection = engine.connect()

from sqlalchemy import Table, Column, Enum, Boolean, Integer, Numeric, Text, Unicode, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref

Base = declarative_base()

class Aisle(Base):
    __tablename__ = 'aisles'
    
    aisle_id = Column('aisle_id', Integer(), autoincrement=True, primary_key=True)
    aisle = Column('aisle', Text)
    
    def __repr__(self):
        return "Aisle(aisle_id={self.aisle_id}, aisle={self.aisle})".format(self=self)

class Department(Base):
    __tablename__ = 'departments'
    
    department_id = Column('department_id', Integer(), autoincrement=True, primary_key=True)
    department = Column('department', Text)
    
    def __repr__(self):
        return "Department(department_id={self.department_id}, department={self.department})".format(self=self)
    
class Product(Base):
    __tablename__ = 'products'
    
    product_id = Column('product_id', Integer(), autoincrement=True, primary_key=True)
    product_name = Column('product_name', Text())
    aisle_id = Column(Integer(), ForeignKey('aisles.aisle_id'))
    department_id = Column(Integer(), ForeignKey('departments.department_id'))
    
    aisle = relationship('Aisle', backref=backref('products', order_by=product_id))
    department = relationship('Department', backref=backref('products', order_by=product_id))
    
    def __repr__(self):
        return u"Product(product_id={self.product_id}, " \
                "product_name={self.product_name}, " \
                "aisle_id={self.aisle_id}, " \
                "department_id={self.department_id})".format(self=self)

class Order(Base):
    __tablename__ = 'orders'
    
    order_id = Column('order_id', Integer(), autoincrement=True, primary_key=True)
    user_id = Column('user_id', Integer())
    order_eval_set = Column('order_eval_set', Enum('prior', 'train', 'test', name='order_eval_set'))
    order_number = Column('order_number', Integer())
    order_dow = Column('order_dow', Integer())
    order_hour_of_day = Column('order_hour_of_day', Integer())
    days_since_prior = Column('days_since_prior', Numeric(), nullable=True)

    def __repr__(self):
        return 'Order(order_id={self.order_id}, ' \
                'user_id={self.user_id}, ' \
                'order_eval_set={self.order_eval_set}, ' \
                'order_number={self.order_number}, ' \
                'order_dow={self.order_dow}, ' \
                'order_hour_of_day={self.order_hour_of_day}, ' \
                'days_since_prior={self.days_since_prior})'.format(self=self)

class LineItem(Base):
    __tablename__ = 'order_products__train'
    
    order_id = Column(Integer(), ForeignKey('orders.order_id'), primary_key=True)
    product_id = Column(Integer(), ForeignKey('products.product_id'), primary_key=True)
    add_to_cart_order = Column('add_to_cart_order', Integer())
    reordered = Column('reordered', Boolean())
    
    order = relationship('Order', backref=backref('line_items', order_by=add_to_cart_order))
    department = relationship('Product', backref=backref('line_items', order_by=add_to_cart_order))
    
    def __repr__(self):
        return 'LineItem(order_id={self.order_id}, ' \
                'product_id={self.product_id}, ' \
                'add_to_cart_order={self.add_to_cart_order}, ' \
                'reordered={self.reordered})'.format(self=self)

Base.metadata.create_all(engine)

In [19]:
# product 24 has a non-ASCII character in it. Print can't understand it.
# What's the alternative to print in ASCII?
product = session.query(Product).filter(Product.product_id == 24).first()

24


In [51]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

from sqlalchemy.sql import func, desc, distinct

# Top 10 departments with the most product listings
for record in session.query(Department.department, func.count().label('product_count')) \
    .join(Product) \
    .group_by(Department.department) \
    .order_by(desc('product_count')) \
    .limit(10):
        print "%20s | %s" % (record.department, record.product_count)  

       personal care | 6563
              snacks | 6264
              pantry | 5371
           beverages | 4365
              frozen | 4007
          dairy eggs | 3449
           household | 3085
        canned goods | 2092
     dry goods pasta | 1858
             produce | 1684


In [52]:
# Top N most ordered items
for record in session.query(Product.product_name, func.count().label('num_orders')) \
    .join(LineItem) \
    .group_by(Product.product_id) \
    .order_by(desc('num_orders')) \
    .limit(10):
        print "%50s | %s" % (record.product_name, record.num_orders)  

                                            Banana | 18726
                            Bag of Organic Bananas | 15480
                              Organic Strawberries | 10894
                              Organic Baby Spinach | 9784
                                       Large Lemon | 8135
                                   Organic Avocado | 7409
                              Organic Hass Avocado | 7293
                                      Strawberries | 6494
                                             Limes | 6033
                               Organic Raspberries | 5546


# Predicting Days Until Next Order Based on the Current Order

Let's build a machine learning model to predict $d = $ days until the next order given only the contents of the current order. We'll represent an order with a feature vector of 0s and 1s. The index of an entry in the vector will correspond to a `product_id`. If the entry is 0, the current order doesn't contain the product. If the entry is 1, the current order contains the product. For example, to represent an order by a customer for `1% Chocolate Milk` and `Kale Apple Greens`, the feature vector would look like

$
order =
\begin{bmatrix}
1 \\
0 \\
0 \\
\vdots \\
1 \\
\vdots \\
0 \\
\end{bmatrix}
\quad
\begin{array}{l}
\text{1% Chocolate Milk} \\
\text{1% Low Fat Cottage Cheese} \\
\text{Acacia Fiber Organic Powder} \\
\vdots \\
\text{Kale Apple Greens} \\
\vdots \\
\text{Zero Calorie Lemon Lime Twist Soda} \\
\end{array}
$

To represent the entire set of orders using for training our ML model, we'll use a $3421083 \times 49688$ matrix

$
orders_{train} =
\overbrace{
\begin{bmatrix}
(order_1)^T \\
(order_2)^T \\
(order_3)^T \\
\vdots \\
(order_{3,421,083})^T \\
\end{bmatrix}
}^{49,688}
$

There are 3,421,083 orders, hence the number of rows. There are 49,688 products, hence the number of columns (or features). Our goal is to learn the weights vector, $\theta \in \mathbb{R}^{49,688 \times 1}$. Each element of this vector is the weight that each product has in determing how many days later the next order will arrive.

Notice that $orders_{train}$ will consist mostly of 0s because the average order contains about 10 out of 49,688 products. This means $orders_{train}$ is a sparse matrix, and perhaps later we can explore taking advantage of this structure to reduce the number of computations to produce $\theta$.

Another thing to notice is that even though we have other information attached to an order such as day of the week, hour of the day, etc., this ML model will assume that those "features" don't matter. In the real world, they probably do, but for now, let's ignore them.

TODO: Write about the label vector, $y$.

Its cost function will be

$
\begin{align}
J(\theta) = \frac{1}{2} \sum_{i = 1}^m \left( h_\theta (x^{(i)}) - y^{(i)}  \right)^2
\end{align}
$

To build the training set, first, let's build one feature vector for an order with its corresponding label.

```
X_tr = orders_train = []
Y_tr = labels_train = []
for user in orders.user_id:
    if not user.has_more_than_one_order:
        continue
    for order, next_order in successive pairs of user.orders:
        if next_order is None:  # all orders except the latest one
            continue
        order_vector = vector of zeroes.
        for product_id in order.products:
            order_vector[product_id - 1] = 1 # use 0-based indexing, hence, subtract 1 from product_id for index
        label = next_order.days_since_prior
        X_tr.append(order_vector)
        Y_tr.append(label)
```

In [92]:
# num_users = session.query(Order.user_id).distinct().count() # 206,209

# order_by(Order.order_number) might not be necessary, but explicitly state it

def preprocess(tup):
    if tup[1] is None:
        second = -1
    else:
        second = int(tup[1])
    return tup[0], second

def orders(user_id):
    order_info_for_user = session.query(Order.order_id, Order.days_since_prior) \
            .filter(Order.user_id == user_id) \
            .order_by(Order.order_number) \
            .all()
    return map(preprocess, order_info_for_user)

def order_products(order_id):
    return session.query(LineItem.product_id).filter(LineItem.order_id == order_id).all()

print order_products(1187899)
# for record in session.query(Order.user_id).distinct().limit(2):
#     order_info_for_user = orders(record.user_id)
#     for (idx, curr_order) in enumerate(order_info_for_user):
#         if (idx + 1 >= len(order_info_for_user)):
#             continue # curr_order is the user's latest order
#         next_order = order_info_for_user[idx + 1]
#         days_until_next_purchase = next_order[1]
#         for product in order.products:
#             # set feature vector
#     print '\n'

[(196,), (25133,), (38928,), (26405,), (39657,), (10258,), (13032,), (26088,), (27845,), (49235,), (46149,)]


In [49]:
# Analyze how many bytes this training set will take up
import numpy as np
import sys

M = 3421083 # number of orders
N = 49688   # number of products, or "features"

# order = np.zeros(N, dtype=np.uint8)
print "Size of 1 order, or feature vector"
print "with np.int: %s bytes" % sys.getsizeof(np.zeros(N, dtype=np.int))
print "with np.uint8: %s bytes" % sys.getsizeof(np.zeros(N, dtype=np.uint8))

# np.uint8 = 8 bits = 1 byte
# np.int = 64 bits = 8 bytes on 64-bit platform

# One np.int is 8 bytes. The numbers in an np.int feature vector take up 8 bytes / feature * 49688 features
print "np.ints take up %d bytes" % (8 * N)
print "np.uint8s take up %d bytes" % (1 * N)

print "Thus, an np array has an overhead of %d == %d bytes." % (49784 - 49688, 397600 - 397504)

print "with np.int, a training set is %d bytes" % (sys.getsizeof(np.zeros(N, dtype=np.int)) * M)
# 1,360,222,600,800 bytes
# 1,360,222,600.8 KB
# 1,360,222.6008 MB
# 1,360.2226008 GB
# 1.3602226008 TB

print "with np.uint8, a training set is %d bytes" % (sys.getsizeof(np.zeros(N, dtype=np.uint8)) * M)
# 170,315,196,072 bytes
# 170,315,196.072 KB
# 170,315.196072 MB
# 170.315196072 GB

# Use a sparse matrix implementation!
import scipy.sparse
x = (np.random.rand(N) * 2).astype(int).astype(float)
m = scipy.sparse.csr_matrix(x)# .toarray()
print m
print sys.getsizeof(m) * M

Size of 1 order, or feature vector
with np.int: 397600 bytes
with np.uint8: 49784 bytes
np.ints take up 397504 bytes
np.uint8s take up 49688 bytes
Thus, an np array has an overhead of 96 == 96 bytes.
with np.int, a training set is 1360222600800 bytes
with np.uint8, a training set is 170315196072 bytes
  (0, 6)	1.0
  (0, 10)	1.0
  (0, 11)	1.0
  (0, 12)	1.0
  (0, 14)	1.0
  (0, 16)	1.0
  (0, 18)	1.0
  (0, 21)	1.0
  (0, 23)	1.0
  (0, 25)	1.0
  (0, 26)	1.0
  (0, 27)	1.0
  (0, 28)	1.0
  (0, 38)	1.0
  (0, 41)	1.0
  (0, 43)	1.0
  (0, 45)	1.0
  (0, 50)	1.0
  (0, 52)	1.0
  (0, 55)	1.0
  (0, 56)	1.0
  (0, 58)	1.0
  (0, 59)	1.0
  (0, 60)	1.0
  (0, 62)	1.0
  :	:
  (0, 49636)	1.0
  (0, 49637)	1.0
  (0, 49639)	1.0
  (0, 49643)	1.0
  (0, 49644)	1.0
  (0, 49645)	1.0
  (0, 49647)	1.0
  (0, 49648)	1.0
  (0, 49650)	1.0
  (0, 49653)	1.0
  (0, 49654)	1.0
  (0, 49655)	1.0
  (0, 49656)	1.0
  (0, 49657)	1.0
  (0, 49658)	1.0
  (0, 49663)	1.0
  (0, 49666)	1.0
  (0, 49668)	1.0
  (0, 49671)	1.0
  (0, 49673)	1.0
  

In [89]:
from scipy.sparse import csr_matrix
sparse_matrix = csr_matrix((1, 10000), dtype=np.uint8)
print sys.getsizeof(sparse_matrix)

64
